In [23]:
import os
import pandas as pd
import pm4py
import numbers
import decimal
import pandas.api.types as ptypes
import numpy as np
from pm4py.algo.filtering.log.cases import case_filter
from pm4py.algo.filtering.pandas.timestamp import timestamp_filter
from pm4py.algo.filtering.log.start_activities import start_activities_filter
from pm4py.algo.filtering.log.end_activities import end_activities_filter
from pm4py.algo.filtering.pandas.variants import variants_filter
from pm4py.objects.log.util import dataframe_utils
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.util import constants
from pm4py.statistics.traces.pandas import case_statistics
from pm4py.algo.filtering.log.attributes import attributes_filter
from numpy.linalg import norm
from pm4py.objects.log.importer.xes import importer as xes_importer
from multiprocessing import Process, Queue, Pool, cpu_count
import multiprocessing
import random
import itertools
import functools
from sklearn.metrics.pairwise import euclidean_distances
from scipy.spatial.distance import cdist, pdist
from scipy.spatial import distance
import math

In [24]:



#convert log file(in csv) to pandas dataframe

"""
QUESTIONS!
importing scheme

1.the major attributes : Case ID, Activity, Complete Timestamp, and org(resource or something)
They should be ordered such that the rest can be considered as case attributes
!!!!!!!this should be done either for xes or csv, also how to parse the given event log file such that
they can be managed in the form Case ID, Acitivity, Timestamp, and resource? is there any scheme to detect which colume
corresponds to which attribute?

2.importing method should deliver also the file path such that this program
can choose either xes or csv file converting

3. what is an empty value exists?

4. organize log file such that
caseid should be the first column
the others resource, timestamp

#It could make the whole process easy
5. allow user to select/organize columns by checking the name
??

Weakness:

1. still depending on the column orders
2. to often used the Case ID
3. def export_file(df, caseids, s_vci, n = 1): find the scheme s.t. 
n indicates unique selection, logarithmic, and division selection

"""

log_csv = pd.read_csv('../eventlogs/ItalianHelpdeskFinal.csv', sep=',')
log_csv = dataframe_utils.convert_timestamp_columns_in_df(log_csv)
param_keys={constants.PARAMETER_CONSTANT_CASEID_KEY: 'Case ID', 
    constants.PARAMETER_CONSTANT_ACTIVITY_KEY: 'Activity', 
            constants.PARAMETER_CONSTANT_TIMESTAMP_KEY: "Complete Timestamp"
           }
event_log = log_converter.apply(log_csv,parameters=param_keys)


#import xes file
#q = xes_importer.apply('../eventlogs/repairExample.xes')

#convert xes to dataframe
#dataframe = log_converter.apply(q, variant=log_converter.Variants.TO_DATA_FRAME)

## 1-Traversing the event log:

In [25]:
#filter variants from the given log_csv
#variants contains seriese objects that consist of Case ID, variant(a sequence of activities), and dtype
variants = case_statistics.get_variants_df(log_csv,
                                          parameters={constants.PARAMETER_CONSTANT_CASEID_KEY: "Case ID",
                                                      constants.PARAMETER_CONSTANT_ACTIVITY_KEY: "Activity"})
variants.describe(include='all')

stats = variants.describe(include='all')

variant_caseid = variants.copy()

#1. Filter cases sharing the same variant

#a. duplicate the case ids as an additional column
variant_caseid['Case IDs'] = variants.index

#b. set variant(sequence of acitvities) as the index
#-> exchange the index from case id to variant
# duplicated Case Ids contain the case Ids, the case Ids in the index is deleted
variant_caseid.set_index('variant', inplace=True)

#c. duplicate variant and store them as an additional column
variant_caseid['variant2'] = variant_caseid.index

#d. collect every case ids sharing the same variant
variant_caseid = list(variant_caseid.groupby("variant2")["Case IDs"].apply(list))

## 2-Distribution Computation:

In [26]:
#copy the dataframe
log_csv2 = log_csv.copy()

#just keep the last activity record
last_activities = log_csv2.drop_duplicates(subset=["Case ID"], keep = "last")

ccols = []
#keep the final returning case id
final_caseid = []

#start for loop to collect categorical column numbers

#drop unnecessary columns such as timestamp and resource
#1:4 means determined index that are not necessary
last_activities = last_activities.drop(list(last_activities.columns)[1:4], axis = 1)

#this function splits the given list a in n parts
#such that splitted sublists have roughly same length
#be used : a : caseids, n : # processes = 4 -> split the list of caseids
def split(a, n):
    
    #the function divmod takes two argument (a1, a2) and returns two results(r1, r2)
    #r1 = a1/a2, r2 = a1 mod a2
    """
    divmod(8, 3) =  (2, 2)
    divmod(7.5, 2.5) =  (3.0, 0.0)
    """
    k, m = divmod(len(a), n)
    return list((a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n)))

#filtersingles filter out all the variant with just 1 cases
#be used : l -> list of caseids
def filtersingles(l):
    
    #empty list to collect all the single variant
    singles = []
    
    #start loop
    for i in l:

        #if the length of the given element is 
        if len(i) == 1:
            
            singles.append(i)
            
    return [x for x in l if x not in singles], singles

#this function uses multiprocessing to apply parallel programming paradigm 
def filtersingles_mp(a_list):
    
    PROCESSES = 4
    nonsingles = []
    singles = []
    
    with multiprocessing.Pool(PROCESSES) as pool:
    
        chunks = split(list(a_list), 4)
    
        results = [pool.apply_async(filtersingles, [chunk]) for chunk in chunks]
        
        for r in results:
            
            nonsingles = nonsingles + r.get()[0]
            singles = singles + r.get()[1]
            
    return nonsingles, singles

ns_vci, s_vci = filtersingles_mp(variant_caseid)

def modifyindex(df):
    
    df = df.set_index('Case ID')
    
    return df

In [27]:
def filter_last_activity_catcols(logdf):
    
    result_df = logdf.copy()
    ccols = []
    
    for c in range(len(list(result_df.columns))):
        
        #check if the column is boolean
        if ptypes.is_bool_dtype(list(result_df.columns)[c]) == True:
            
            #then multiply 1 to whole column such that they can be numeric
            #used if else such that each column can be checked with on condition 
            result_df.iloc[:, c] = result_df.iloc[:, c].apply(lambda x: x*1)
        
        #check if a column is numerical
        elif ptypes.is_numeric_dtype(list(result_df.columns)[c]) != True:
            
            #else go into the catcols list
            ccols.append(c)
            
        else :
        
            continue
    
    #result_df keeps just the last activities of each traces
    #keep the index of categorical colums
    return result_df, ccols

last_activities, ccols = filter_last_activity_catcols(last_activities)

#this function returns the dataframe part regarding the given caseid
#be used : splits the log dataframe corresponding the given caseid
def filterdf(logdf, caseid):

    return logdf.loc[logdf["Case ID"].isin(caseid)]

def splitdf(logdf, caseids):

    dfs = []
    
    for ci in caseids:
        
        dfs.append(filterdf(logdf, ci))
        
    return dfs

#this function applies splitdf in multithreading
#logdf : dataframe to be splitted
#caseids : list of caseid (variant_caseid) will be used
def splitdf_mp(logdf, caseids):
    
    # the number of processes
    PROCESSES = 4
    
    #list of dataframes to be returned
    dfs = []
    
    with multiprocessing.Pool(PROCESSES) as pool:
        
        #split caseid into 4 
        chunks = split(list(caseids), 4)
        
        results = [pool.apply_async(splitdf, (logdf, chunk)) for chunk in chunks]
    
        for r in results:
            
            dfs.append(r.get())
            
    return dfs

#subset of dfs corresponding their single variant
#each single element in the list is dataframe
dfs_variant = splitdf_mp(last_activities, ns_vci)

In [29]:
#One hot encoding
#now apply one hot encoding using every categorical colum from the catcols     
#check every list of case id of each variant

#this function applies one hot encoding to the given dataframe regarding the given column names
#df : dataframe to be the one hot encoding applied
#columns : columnnames(should be categorical columns)
def one_hot_encoding(df, columns):
    
    encoded = []
    first_column = df.iloc[:, 0]
    encoded.append(first_column)
    #
    
    for c in range(1,len(columns)):
    
        #one hot encoding of each colum
        encoded.append(pd.get_dummies(df.iloc[:, c], prefix = df.columns[c]))
        
    return pd.concat(encoded, axis=1)

def map_one_hot_encoding(dfs, columns):
    
    one_hot_encoded = []
    
    for i in range(len(dfs)):
        
        one_hot_encoded.append(list(map(functools.partial(one_hot_encoding, columns=columns), dfs[i])))
        
    return one_hot_encoded

dfs_variant = map_one_hot_encoding(dfs_variant, ccols)


In [33]:

def euclidean_distance(df):
    
    #set the Case Id(first column) as index
    df = modifyindex(df)
    #remove all columns having either 1 or 0(do not need to compute in the euclidean distance)  
    df = df.loc[:,~df.eq(0).all()]
    df = df.loc[:,~df.eq(1).all()]

    #add the dist column containing the euclidean distance to every rows
    df['dist']=pd.Series(np.sum(distance.cdist(df.iloc[:,1:], df.iloc[:,1:], metric='euclidean'), axis = 1), index = df.index)
    
    #sort this small dataframe by the dist
    df = df.sort_values(by=['dist'])
    
    #rank this dataframe by the distance
    #frame['sum_order'] = frame['sum'].rank()
    
    df['dist_order'] = df['dist'].rank()
    
    #print(df)
    
    return df['dist_order']

def euclidean_distance_list(dfs):
    
    new_dfs = []
    
    for df in dfs:
        
        new_dfs.append(euclidean_distance(df))
        
    return new_dfs
        
        
#this function computes the euclidean distance using multiprocessing
def euclidean_distance_mp(dfs):
    
    # the number of processes
    PROCESSES = 4
    
    with multiprocessing.Pool(PROCESSES) as pool:
        
        results = pool.map(euclidean_distance_list, dfs)
    
        return results
    
    #return dfs

#k = euclidean_distance(dfs_variant[0][0])
#k = one_hot_encoding(dfs_variant[0][0], ccols)

#one_hot_encoded = map_one_hot_encoding(dfs_variant, ccols)
r = euclidean_distance_mp(dfs_variant)

type(r[0][0])

pandas.core.series.Series

In [40]:
#returns the list of ones with length n
#whereby n means the number of dataframes in a list
#aims def final_caseids(tp):
def unique_selection_df(df):
    
    return [df, 1]

def unique_selection_list(dfs):
    
    return list(map(unique_selection_df, dfs))

def unique_selection(dfs):
        
    return list(map(unique_selection_list, dfs))

#this function returns Log_2(the number of cases of each variant)
def logarithmic_selection(df):
    
    index = df.index
    return round(math.log2(len(index)))

#this function returns Log_2(every number of cases of each variant)
def logarithmic_selection_list(dfs):
    
    nums = []
    
    for df in dfs:
        
        nums.append(logarithmic_selection(df))
        
    return list(zip(dfs, nums))

#this function returns Log_2(every number of cases of every variant)
def logarithmic_selection_df(dfs):
    
    return list(map(logarithmic_selection_list, dfs))

#this function returns the number of cases of each variant / 2
def division_selection(df):
    
    index = df.index
    return round(len(index)/2)

#this function returns the number of every cases of each variant / 2
def division_selection_list(dfs):
    
    nums = []
    
    for df in dfs:
        
        nums.append(division_selection(df))
        
    return list(zip(dfs, nums))

#this function returns all the number of cases of every variant / 2
def division_selection_df(dfs):
    
    return list(map(division_selection_list, dfs))

In [57]:
#return the final case ids
#depending on the selection method, n has different values s.t
# unique selection n -> 1 (default setting)
# logarithmic selection n -> log2( number of traces of each variant)
# division selection n -> round(number of traces of each variant / 2)

#this function takes a tuple of 
# (pandas.core.series.Series, n)
# n : the number of cases that should be returned
def final_caseids(tp):
    
    df = tp[0].to_frame()
    return list(df.head(tp[1]).index)
    
#dfs means the list of tuples of (dataframe, n)
def final_caseids_list(dfs):
    
    return [final_caseids(df) for df in dfs]

#r is the final all ranked caseids

def flatten(t):
    return [item for sublist in t for item in sublist]

def final_caseids_mp(dfs, n = 1):
    
    # the number of processes
    PROCESSES = 4
    
    #L = pool.starmap(func, [(1, 1), (2, 1), (3, 1)])
    
    with multiprocessing.Pool(PROCESSES) as pool:
        
        #.starmap(f_sum, data)
        results = pool.map(final_caseids_list, dfs)
        
    results = flatten(flatten(results))
    
    #results = [item for sublist in dfs for item in sublist]
    return results

['Case 1535',
 'Case 2525',
 'Case 874',
 'Case 2684',
 'Case 3456',
 'Case 3119',
 'Case 4197',
 'Case 725',
 'Case 2649',
 'Case 7',
 'Case 2853',
 'Case 474',
 'Case 1641',
 'Case 4387',
 'Case 798',
 'Case 4085',
 'Case 1095',
 'Case 2278',
 'Case 3297',
 'Case 777',
 'Case 1843',
 'Case 4530',
 'Case 3803',
 'Case 2523',
 'Case 3961',
 'Case 75',
 'Case 1098',
 'Case 912',
 'Case 1706',
 'Case 3759',
 'Case 260',
 'Case 2798',
 'Case 363',
 'Case 1211',
 'Case 2136',
 'Case 3788',
 'Case 2527',
 'Case 3084',
 'Case 803',
 'Case 3302',
 'Case 2778',
 'Case 3625',
 'Case 3359',
 'Case 1422',
 'Case 246',
 'Case 485',
 'Case 4050',
 'Case 4065',
 'Case 1981',
 'Case 4392',
 'Case 2309',
 'Case 3001',
 'Case 1929',
 'Case 642',
 'Case 1118',
 'Case 3235',
 'Case 406',
 'Case 214',
 'Case 759',
 'Case 2086',
 'Case 1000',
 'Case 3757',
 'Case 2636',
 'Case 189',
 'Case 889',
 'Case 2011',
 'Case 231',
 'Case 1633',
 'Case 4287',
 'Case 974',
 'Case 2272',
 'Case 2107',
 'Case 451',
 'C

In [58]:
def export_file(df, caseids, s_vci, n = 1):
    
    #2 indicates that it is logarithmic
    #it is just for now!!!!
    if n != 2:
        
        caseids = caseids + s_vci
    
    export_csv = df[df['Case ID'].isin(caseids)]
    export_csv.to_csv('exported.csv', index = True, header=True)
    

## How to execute all the functions?


In [60]:
#separate variants between
#ns_vci : NonSingle Variant Case ID
#s_vci : Single Variant Case ID
ns_vci, s_vci = filtersingles_mp(variant_caseid)

#extract just the very last row of each cases and separate Categorical Columns (ccols)
last_activities, ccols = filter_last_activity_catcols(last_activities)

#subset of dfs corresponding their single variant
#each single element in the list is dataframe
dfs_variant = splitdf_mp(last_activities, ns_vci)

#One hot encoding
dfs_variant = map_one_hot_encoding(dfs_variant, ccols)

#euclidean distance is computed and ranked by this
r = euclidean_distance_mp(dfs_variant)

"""

now we should select which method we want to use
if we choose unique selection,

unique_selection(dfs)

if logarithmic
logarithmic_selection_df

if division selection

division_selection_df

"""
r = unique_selection(r)

#extract final case ids that passed the euclidean_distance_mp
final_caseid_list = final_caseids_mp(r)

#export the csv file corresponding to the imported csv file
export_file(log_csv, r, s_vci)

In [29]:
def blu(a):
    
    print(a)
    
loo = (3,4)
    
blu(loo)

(3, 4)
